<a href="https://colab.research.google.com/github/witsarutsarai12-Academic/128-356-Big-Data/blob/main/BigData_Week3_SQL_DuckDB_to_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Data : SQL Analytics ด้วย DuckDB → Spark SQL

> เป้าหมายวันนี้: ใช้ **SQL** เพื่อ “ถามข้อมูล” จากไฟล์ใหญ่ (CSV/Parquet) ด้วย **DuckDB** และเข้าใจแนวคิด performance แบบที่ย้ายไปใช้ **Spark SQL** ต่อได้

**ใช้ได้ทั้ง**: Google Colab (แนะนำ) / Local Jupyter  

---


##  รู้จัก Bash Commands เบื้องต้น

<img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/bash_cmd.png?raw=1"  width="700" style="max-width:100%; height:auto;"/>

### ทำไมต้องเรียนรู้ Command Line?

ในโลกของ Data Engineering และ Big Data เราจะต้องทำงานกับ:
- **Remote Servers** (เซิร์ฟเวอร์ที่อยู่ใน Cloud เช่น AWS EC2, Google Cloud)
- **Data Lakes** (ข้อมูลที่เก็บบน S3, HDFS)
- **Cluster Computing** (Spark Cluster ที่มีหลายเครื่อง)

ในสถานการณ์เหล่านี้ เรา**ไม่มี GUI** (Graphical User Interface) ให้คลิกเหมือนบน Windows/Mac
เราต้องใช้ **Command Line Interface (CLI)** หรือที่เรียกว่า **Terminal/Bash** ในการทำงาน

### ประโยชน์ของ Bash Commands:
1. **ตรวจสอบไฟล์ก่อนโหลด**: ดูว่าไฟล์มีขนาดเท่าไหร่ มีกี่บรรทัด หน้าตาข้อมูลเป็นอย่างไร
2. **ประหยัดเวลา**: ไม่ต้องรอ Download ไฟล์ขนาดใหญ่มาดูในเครื่อง
3. **Automation**: เขียน Script เพื่อทำงานซ้ำ ๆ อัตโนมัติ
4. **ทำงานกับ Big Data**: ไฟล์ขนาด GB-TB ไม่สามารถเปิดด้วย Excel ได้

### คำสั่งพื้นฐานที่ต้องรู้:

#### 1. `pwd` (Print Working Directory)
- **ความหมาย**: แสดงตำแหน่งปัจจุบันที่เราอยู่
- **ตัวอย่าง**: `/home/username/data`
- **เมื่อไหร่ใช้**: เมื่อหลงทางว่าตอนนี้อยู่โฟลเดอร์ไหน

#### 2. `ls` (List)
- **ความหมาย**: แสดงรายชื่อไฟล์และโฟลเดอร์ในตำแหน่งปัจจุบัน
- **ตัวอย่าง**: `ls` → แสดงชื่อไฟล์
- **Options ที่ใช้บ่อย**:
  - `ls -l` → แสดงแบบละเอียด (สิทธิ์, ขนาด, วันที่)
  - `ls -lh` → แสดงขนาดไฟล์ในรูปแบบที่คนอ่านง่าย (MB, GB แทนที่จะเป็น bytes)
  - `ls -a` → แสดงไฟล์ที่ซ่อนอยู่ (ขึ้นต้นด้วย `.`)

#### 3. `cd` (Change Directory)
- **ความหมาย**: เปลี่ยนโฟลเดอร์
- **ตัวอย่าง**:
  - `cd data` → เข้าไปในโฟลเดอร์ data
  - `cd ..` → ย้อนกลับไปโฟลเดอร์ก่อนหน้า
  - `cd ~` → กลับไปที่ Home directory

#### 4. `head` (แสดงบรรทัดแรก)
- **ความหมาย**: แสดง 10 บรรทัดแรกของไฟล์ (Default)
- **ตัวอย่าง**:
  - `head sample_data/california_housing_train.csv` → ดู 10 บรรทัดแรก
  - `head -n 5 sample_data/california_housing_train.csv` → ดู 5 บรรทัดแรก
- **เมื่อไหร่ใช้**: ต้องการดูว่าไฟล์มี Header อะไรบ้าง หรือข้อมูลหน้าตาเป็นอย่างไร

#### 5. `tail` (แสดงบรรทัดท้าย)
- **ความหมาย**: แสดง 10 บรรทัดสุดท้ายของไฟล์
- **ตัวอย่าง**: `tail -n 20 sample_data/california_housing_train.csv` → ดู 20 บรรทัดสุดท้าย
- **เมื่อไหร่ใช้**: ดูว่าไฟล์มีข้อมูลครบหรือไม่ หรือดู Log ล่าสุด

#### 6. `cat` (Concatenate)
- **ความหมาย**: แสดงเนื้อหาทั้งหมดของไฟล์
- **ตัวอย่าง**: `cat small_file.txt`
- **⚠️ คำเตือน**: อย่าใช้กับไฟล์ใหญ่! จอจะท่วมและอาจค้าง

#### 7. `wc` (Word Count)
- **ความหมาย**: นับจำนวนบรรทัด, คำ, bytes
- **ตัวอย่าง**:
  - `wc -l sample_data/california_housing_train.csv` → นับจำนวนบรรทัด (มีข้อมูลกี่แถว)
  - `wc -c sample_data/california_housing_train.csv` → นับจำนวน bytes (ขนาดไฟล์)

#### 8. `grep` (ค้นหาข้อความ)
- **ความหมาย**: ค้นหาบรรทัดที่มีคำที่ระบุ
- **ตัวอย่าง**: `grep "THEFT" sample_data/california_housing_train.csv` → แสดงเฉพาะบรรทัดที่มีคำว่า THEFT

### การใช้งานใน Jupyter/Colab

ใน Jupyter Notebook หรือ Google Colab เราสามารถรันคำสั่ง Bash ได้โดยเติม `!` ข้างหน้า:

```python
!pwd                    # ดูว่าอยู่โฟลเดอร์ไหน
!ls -lh                 # ดูไฟล์ในโฟลเดอร์
!head -n 5 sample_data/california_housing_train.csv    # ดู 5 บรรทัดแรก
!wc -l sample_data/california_housing_train.csv        # นับจำนวนบรรทัด
```

### ทำไมต้องเช็คก่อนโหลดข้อมูล?

**สถานการณ์จริง**:
1. คุณเจอไฟล์ `sales_data.csv` บน S3
2. ถ้า Download มาเลย อาจพบว่า:
   - ไฟล์ใหญ่ 50 GB (ใช้เวลา Download นาน)
   - ไฟล์เสีย (มีแต่ Header ไม่มีข้อมูล)
   - Format ผิด (คั่นด้วย `;` แทน `,`)
3. ถ้าใช้ `head` ดูก่อน จะรู้ทันทีว่าไฟล์มีปัญหาหรือไม่

> **💡 Best Practice**: ก่อน Load ข้อมูลด้วย Python/SQL ให้ใช้ Bash ตรวจสอบก่อนเสมอ!


In [50]:
# ตัวอย่างการใช้ Bash Commands ใน Jupyter/Colab
# (ลบ # ออกเพื่อรัน)

# !pwd                    # ดูว่าอยู่โฟลเดอร์ไหน
# !ls -lh                 # ดูไฟล์ทั้งหมดพร้อมขนาด
# !head -n 5 sample_data/california_housing_train.csv    # ดู 5 บรรทัดแรกของไฟล์
# !wc -l sample_data/california_housing_train.csv        # นับจำนวนบรรทัดทั้งหมด
# !grep 100 sample_data/california_housing_train.csv | head -n 3  # ค้นหาคำว่า THEFT และแสดง 3 บรรทัดแรก


## 0.1) History & Importance of SQL

<img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/sql_timeline.png?raw=1" width="700" style="max-width:100%; height:auto;">

<img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/sql_evolution.png?raw=1" width="700" style="max-width:100%; height:auto;">


### ประวัติย่อของ SQL (Structured Query Language)

SQL เป็นภาษาที่ใช้ในการจัดการข้อมูลในฐานข้อมูลเชิงสัมพันธ์ (Relational Database) มาตั้งแต่ยุค 1970 และยังคงเป็นมาตรฐานอุตสาหกรรมจนถึงปัจจุบัน

#### Timeline สำคัญ:
- **1970**: Edgar Codd นักวิจัยจาก IBM เผยแพร่บทความ *"A Relational Model of Data for Large Shared Data Banks"* ซึ่งเป็นรากฐานของ Relational Database
- **1974-1975**: IBM พัฒนา SEQUEL (Structured English Query Language) ซึ่งต่อมาเปลี่ยนชื่อเป็น SQL เพื่อใช้กับระบบ System R
- **1979**: Oracle Corporation (ชื่อเดิม Relational Software Inc.) เปิดตัว Oracle V2 ซึ่งเป็น RDBMS เชิงพาณิชย์ตัวแรกของโลก
- **1986**: ANSI (American National Standards Institute) ประกาศให้ SQL เป็นมาตรฐานอย่างเป็นทางการ
- **1987**: ISO (International Organization for Standardization) รับรอง SQL เป็นมาตรฐานสากล
- **ปัจจุบัน**: SQL ยังคงเป็นภาษาหลักในการจัดการข้อมูล ไม่ว่าจะเป็น RDBMS แบบดั้งเดิม, Data Warehouse, หรือแม้แต่ระบบ Big Data

### วิวัฒนาการของ SQL Engine

SQL เป็นเพียง **ภาษา** (Language) ที่เราใช้เขียนคำสั่ง แต่สิ่งที่ทำงานจริงคือ **Engine** หรือ **Database Management System (DBMS)** ซึ่งมีวิวัฒนาการตามความต้องการของยุคสมัย:

#### 1. RDBMS (Row-oriented) - ยุคแรก
- **ตัวอย่าง**: Oracle, PostgreSQL, MySQL, SQL Server
- **จุดเด่น**: เหมาะกับงาน **OLTP (Online Transaction Processing)** เช่น ระบบธนาคาร, ระบบ E-commerce
- **การจัดเก็บ**: เก็บข้อมูลเป็น **แถว** (Row) ทำให้การ Insert/Update/Delete เร็ว
- **ข้อจำกัด**: การวิเคราะห์ข้อมูลขนาดใหญ่ (Analytical Query) ช้า เพราะต้องอ่านทั้งแถวแม้ว่าจะใช้แค่บางคอลัมน์

#### 2. Data Warehouse (Column-oriented) - ยุคการวิเคราะห์
- **ตัวอย่าง**: Teradata, Netezza, Vertica, Amazon Redshift
- **จุดเด่น**: เหมาะกับงาน **OLAP (Online Analytical Processing)** เช่น รายงานยอดขาย, การวิเคราะห์แนวโน้ม
- **การจัดเก็บ**: เก็บข้อมูลเป็น **คอลัมน์** (Column) ทำให้การอ่านข้อมูลเฉพาะคอลัมน์ที่ต้องการเร็วมาก
- **เทคนิค**: รองรับ Compression ได้ดี เพราะข้อมูลในคอลัมน์เดียวกันมักมีรูปแบบคล้ายกัน

#### 3. Big Data Era (Distributed) - ยุคข้อมูลขนาดใหญ่
- **Apache Hive**: แปลง SQL เป็น MapReduce jobs ทำงานบน Hadoop (ช้า แต่รองรับข้อมูลขนาด Petabyte)
- **Spark SQL**: ประมวลผลแบบ In-Memory (เก็บข้อมูลในแรม) ทำให้เร็วกว่า Hive หลายเท่า รองรับทั้ง Batch และ Streaming
- **Presto/Trino**: Interactive Query Engine สามารถ Query ข้อมูลจากหลายแหล่ง (S3, MySQL, Postgres) พร้อมกันได้
- **จุดเด่น**: กระจายงานไปทำบนเครื่องหลายร้อย-หลายพันเครื่อง (Distributed Computing)

#### 4. Modern In-Process Analytics - ยุคปัจจุบัน
- **DuckDB**: ฐานข้อมูล OLAP ที่รันในเครื่องเดียว (In-Process) เหมือน SQLite แต่เน้นการวิเคราะห์
- **จุดเด่น**: ไม่ต้องติดตั้ง Server, รวดเร็วมาก, เหมาะกับ Data Scientist ที่ทำงานบน Laptop
- **ข้อจำกัด**: ถูกจำกัดด้วย RAM ของเครื่องเดียว (ไม่เหมาะกับข้อมูล TB-PB)

### ทำไม SQL ถึงสำคัญกับ Data Engineer/Scientist?

#### 1. Universal Interface (ภาษาสากล)
- ไม่ว่าข้อมูลจะอยู่ใน CSV, Parquet, PostgreSQL, MySQL, S3 Data Lake หรือแม้แต่ Google Sheets
- เราสามารถใช้ SQL เพื่อ Query ข้อมูลได้เกือบทั้งหมด
- **ตัวอย่าง**: DuckDB สามารถ Query ไฟล์ Parquet บน S3 ได้โดยตรงโดยไม่ต้อง Download

#### 2. Declarative Programming (บอกว่าต้องการอะไร ไม่ต้องบอกว่าทำอย่างไร)
- เราแค่เขียนว่า `SELECT name, SUM(sales) FROM orders WHERE year = 2023 GROUP BY name`
- Engine จะเป็นคนหาวิธีที่เร็วที่สุด (Query Optimization) เช่น:
  - ใช้ Index ถ้ามี
  - อ่านเฉพาะ Partition ที่จำเป็น
  - ทำ Parallel Processing
- ถ้าเขียนเป็น Python Loop เราต้องคิดเองทุกอย่าง และมักจะช้ากว่า

#### 3. Scalability (ขยายได้ตามขนาดข้อมูล)
- SQL เดียวกัน สามารถรันบน:
  - SQLite (ข้อมูล MB-GB บนมือถือ)
  - PostgreSQL (ข้อมูล GB-TB บน Server)
  - Spark SQL (ข้อมูล TB-PB บน Cluster)
- เราไม่ต้องเขียนโค้ดใหม่ เพียงแค่เปลี่ยน Engine

#### 4. Industry Standard (มาตรฐานอุตสาหกรรม)
- ทุกคนในทีม Data (Analyst, Engineer, Scientist) รู้จัก SQL
- ง่ายต่อการ Collaborate และ Maintain โค้ด
- มี Community และ Resources มากมาย

<img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/sql_engine_types.png?raw=1" width="700" style="max-width:100%; height:auto;">

## 0) Setup & ติดตั้ง DuckDB (Colab)

> ถ้าเป็น Colab ให้รัน cell นี้ก่อน


In [53]:
# ติดตั้ง (Colab)
!pip -q install duckdb pandas pyarrow

import duckdb, pandas as pd, time, os
print("DuckDB version:", duckdb.__version__)

DuckDB version: 1.3.2


## 1) เตรียมไฟล์ข้อมูล

- ถ้าไฟล์อยู่ในโฟลเดอร์เดียวกับ notebook ให้ใส่ชื่อไฟล์ตรง ๆ
- ถ้าอยู่คนละที่ ให้ใส่ path เต็ม

> **หมายเหตุ**: ถ้ายังไม่มีไฟล์ในเครื่อง/Colab ให้ผู้สอนอัปโหลด หรือ mount Drive ก่อน


In [55]:
# # ใช้ Dataset California Housing ที่มีอยู่แล้วใน Colab
# CSV_PATH = "sample_data/california_housing_train.csv"
# PARQUET_PATH = "housing.parquet"
# df.head()
# df.to_parquet(PARQUET_PATH)

# # import pandas as pd
# # import os

# # สร้างไฟล์ Parquet จาก CSV เพื่อใช้ในการทดสอบ Performance (ถ้ายังไม่มี)
# # if not os.path.exists(PARQUET_PATH) and os.path.exists(CSV_PATH):
# #     print(f"🔨 กำลังสร้าง {PARQUET_PATH} จาก CSV...")
# #     pd.read_csv(CSV_PATH).to_parquet(PARQUET_PATH)
# #     print("✅ สร้างไฟล์ Parquet เสร็จสิ้น")

# # print("CSV exists:", os.path.exists(CSV_PATH))
# # print("Parquet exists:", os.path.exists(PARQUET_PATH))


In [ ]:
PARQUET_PATH = "house"

## 2) สร้าง Connection + สร้าง VIEW

แนวคิด:
- เราจะ query ไฟล์โดยตรง (ไม่ต้อง import เข้า DB)
- สร้าง `VIEW` เพื่อเขียน SQL สั้นลง


In [56]:
# Parquet view
if os.path.exists(PARQUET_PATH):
    con.execute(f"""
        CREATE OR REPLACE VIEW housing_parquet AS
        SELECT * FROM read_parquet('{PARQUET_PATH}');
    """)
    print("✅ created view: housing_parquet")
else:
    print("⚠️ ไม่พบไฟล์ PARQUET_PATH")


✅ created view: housing_parquet


## 3) ดูตัวอย่างข้อมูลอย่างปลอดภัย (LIMIT)

> **ห้าม** `SELECT *` โดยไม่ใส่ `LIMIT` เวลาข้อมูลใหญ่


In [57]:
# เลือก view ที่มีอยู่จริง (parquet ถ้ามี จะ prefer เพราะเร็ว)
available = [v[0] for v in con.execute("SHOW TABLES").fetchall()]
table = "housing_parquet" if "housing_parquet" in available else ("housing_csv" if "housing_csv" in available else None)

assert table is not None, "ต้องมีอย่างน้อย housing data หรือ housing.parquet"

con.execute(f"SELECT * FROM {table} LIMIT 10").df()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
5,-114.58,33.63,29.0,1387.0,236.0,671.0,239.0,3.3438,74000.0
6,-114.58,33.61,25.0,2907.0,680.0,1841.0,633.0,2.6768,82400.0
7,-114.59,34.83,41.0,812.0,168.0,375.0,158.0,1.7083,48500.0
8,-114.59,33.61,34.0,4789.0,1175.0,3134.0,1056.0,2.1782,58400.0
9,-114.60,34.83,46.0,1497.0,309.0,787.0,271.0,2.1908,48100.0


In [58]:
# T000: เติม sql
sqll = f"""
SELECT * AS n_rows
FROM {table}
LIMIT 10
"""

#sql2: เลือกคอลลัมม์ 4 ตัว + LIMIT5
sql2 = f"""
SELECT longitude, latitude, housing_median_age, total_rooms
FROM {table}
LIMIT 5
"""

#รับ
try:
    print(con.execute(sqll).df())
    display(con.execute(sql2).df())
except Exception as e:
    print("❌ Error:", e)

   n_rows  n_rows_1  n_rows_2  n_rows_3  n_rows_4  n_rows_5  n_rows_6  \
0 -114.31     34.19      15.0    5612.0    1283.0    1015.0     472.0   
1 -114.47     34.40      19.0    7650.0    1901.0    1129.0     463.0   
2 -114.56     33.69      17.0     720.0     174.0     333.0     117.0   
3 -114.57     33.64      14.0    1501.0     337.0     515.0     226.0   
4 -114.57     33.57      20.0    1454.0     326.0     624.0     262.0   
5 -114.58     33.63      29.0    1387.0     236.0     671.0     239.0   
6 -114.58     33.61      25.0    2907.0     680.0    1841.0     633.0   
7 -114.59     34.83      41.0     812.0     168.0     375.0     158.0   
8 -114.59     33.61      34.0    4789.0    1175.0    3134.0    1056.0   
9 -114.60     34.83      46.0    1497.0     309.0     787.0     271.0   

   n_rows_7  n_rows_8  
0    1.4936   66900.0  
1    1.8200   80100.0  
2    1.6509   85700.0  
3    3.1917   73400.0  
4    1.9250   65500.0  
5    3.3438   74000.0  
6    2.6768   82400.0  
7   

,longitude,latitude,housing_median_age,total_rooms
0,-114.31,34.19,15.0,5612.0
1,-114.47,34.40,19.0,7650.0
2,-114.56,33.69,17.0,720.0
3,-114.57,33.64,14.0,1501.0
4,-114.57,33.57,20.0,1454.0


## 4) ตรวจ schema / ชนิดข้อมูล

> ถ้า type ผิด (เช่น year เป็นข้อความ) จะทำให้ sort/เงื่อนไขผิดได้


In [ ]:
con.execute(f"DESCRIBE SELECT * FROM {table}").df()

### SQL Logical Order of Execution (ลำดับการทำงานจริง)

ภาพด้านล่างแสดงลำดับที่ Database Engine ทำงานจริง ซึ่งต่างจากลำดับที่เราเขียน (Lexical Order)

![SQL Logical Order](https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/sql_logical_order_v2.png?raw=1)

> **Note**: สังเกตว่า `SELECT` ทำงานเกือบสุดท้าย! นั่นเป็นเหตุผลว่าทำไมเราถึงใช้ Alias (ชื่อเล่น) ที่ตั้งใน SELECT ไปใช้ใน WHERE ไม่ได้


In [59]:
# ลองรันก่อน ถ้าคอลัมน์ไม่ตรง ให้แก้ชื่อคอลัมน์ใน SQL
con.execute(f"""
    SELECT median_house_value, housing_median_age, total_rooms
    FROM {table}
    WHERE housing_median_age > 50
    LIMIT 20
""").df()


,median_house_value,housing_median_age,total_rooms
0,128600.0,52.0,3419.0
1,270800.0,52.0,2606.0
2,300000.0,52.0,1506.0
3,331400.0,52.0,1484.0
4,83300.0,52.0,280.0
5,47500.0,52.0,204.0
6,163400.0,52.0,1969.0
7,185300.0,52.0,1512.0
8,213100.0,52.0,2676.0
9,221100.0,52.0,1911.0


### ✅ โจทย์ 1 (Easy)
1) นับจำนวนแถวทั้งหมดในตาราง  
2) แสดง 5 แถวแรก (เลือกเฉพาะ 3–5 คอลัมน์พอ)

เติม SQL ในช่องว่างด้านล่าง


In [60]:
# TODO: เติม SQL ให้ครบ
sql1 = f"""
SELECT ________ AS n_rows
FROM {table}
"""

# sql2: เลือกคอลัมน์ 3-5 ตัว + LIMIT 5
sql2 = f"""
SELECT ________, ________, ________
FROM {table}
LIMIT 5
"""

# รัน
try:
    print(con.execute(sql1).df())
    display(con.execute(sql2).df())
except Exception as e:
    print("❌ Error:", e)

❌ Error: Binder Error: Referenced column "________" not found in FROM clause!
Candidate bindings: "housing_median_age"

LINE 2: SELECT ________ AS n_rows
               ^


## 6) Aggregation: GROUP BY / ORDER BY

แนวคิด: Big Data ไม่ดูทีละแถว → สรุปภาพรวม


In [ ]:
con.execute(f"""
    SELECT
      housing_median_age,
      COUNT(*) AS total_houses,
      AVG(median_house_value) AS avg_price
    FROM {table}
    GROUP BY housing_median_age
    ORDER BY total_houses DESC
    LIMIT 10
""").df()


### ✅ โจทย์ 2 (Medium)
1) หา **housing_median_age** ที่มีจำนวนบ้านมากสุด 5 อันดับ  
2) สำหรับ housing_median_age อันดับ 1: หา **avg(median_house_value)** ของอายุนั้น  

ทำเป็น 2 query (ยังไม่ต้อง subquery)


In [61]:
# TODO: เติม SQL
sql_top_age = f"""
SELECT housing_median_age, COUNT(*) AS total_houses
FROM {table}
GROUP BY housing_median_age
ORDER BY total_houses DESC
LIMIT 5
""".strip()

sql_price_by_age = f"""
-- แก้ค่า 52 (อายุบ้าน) ให้เป็นค่าที่คุณอยากรู้
SELECT housing_median_age, AVG(median_house_value) AS avg_price
FROM {table}
WHERE housing_median_age = 52
GROUP BY housing_median_age
""".strip()

try:
    display(con.execute(sql_top_age).df())
    display(con.execute(sql_price_by_age).df())
except Exception as e:
    print(e)


,housing_median_age,total_houses
0,52.0,1052
1,36.0,715
2,35.0,692
3,16.0,635
4,17.0,576


,housing_median_age,avg_price
0,52.0,277368.20057


## 7) Performance Lab: CSV vs Parquet + Timing

เราจะวัดเวลา query เดียวกันบน CSV และ Parquet (ถ้ามีทั้งคู่)


In [62]:
def time_query(sql: str, repeat: int = 3) -> float:
    times = []
    for _ in range(repeat):
        t0 = time.time()
        con.execute(sql).fetchall()
        times.append(time.time() - t0)
    return min(times)

# ตัวอย่าง query ที่มักได้ประโยชน์จาก Parquet (filter + count)
q = """SELECT COUNT(*) FROM {table} WHERE housing_median_age > 10 AND median_house_value > 100000""".format(table=table)
print("table:", table)
print("query:", q)
print("time (best of 3):", time_query(q), "sec")


table: housing_parquet
query: SELECT COUNT(*) FROM housing_parquet WHERE housing_median_age > 10 AND median_house_value > 100000
time (best of 3): 0.002920866012573242 sec


### ✅ โจทย์ 3 (Analysis)
1) ลอง query แบบเลือกคอลัมน์น้อย (เช่น COUNT หรือเลือก 1 คอลัมน์)  
2) ลอง query แบบเลือกคอลัมน์เยอะ (เช่น SELECT 6–10 คอลัมน์)  
3) เปรียบเทียบเวลา แล้วตอบสั้น ๆ: **ทำไมเวลาอาจต่างกัน?**


## 8) EXPLAIN / EXPLAIN ANALYZE (ดู execution plan)

> เห็นว่าระบบจะ scan/ filter/ aggregate ยังไง


In [63]:
con.execute(f"""
EXPLAIN SELECT COUNT(*)
FROM {table}
WHERE housing_median_age > 20
""").df()


,explain_key,explain_value
0,physical_plan,┌───────────────────────────┐\n│ UNGROUPED_...


## 9) JOIN + CTE

เราจะสร้างตาราง lookup เล็ก ๆ (dimension) แล้ว JOIN

<img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/sql_join_cte.png?raw=1" width="700" />

In [64]:
age_map = pd.DataFrame({
    "housing_median_age": range(1, 100),
    "category": ["New" if x < 20 else ("Mid" if x < 40 else "Old") for x in range(1, 100)]
})
con.register("age_map", age_map)

con.execute("SELECT * FROM age_map LIMIT 5").df()


,housing_median_age,category
0,1,New
1,2,New
2,3,New
3,4,New
4,5,New


In [65]:
con.execute(f"""
    SELECT
      m.category,
      COUNT(*) AS total_houses,
      AVG(h.median_house_value) AS avg_price
    FROM {table} h
    JOIN age_map m
      ON h.housing_median_age = m.housing_median_age
    GROUP BY m.category
    ORDER BY avg_price DESC
""").df()


,category,total_houses,avg_price
0,Old,3419,228626.052354
1,Mid,8755,206404.182753
2,New,4826,193819.812889


### ✅ โจทย์ 4 (Medium–Hard): CTE 2 ชั้น
- `base`: กรองเฉพาะราคาบ้าน > 200000  
- `agg`: รวมจำนวนต่อ housing_median_age  
- จากนั้นเลือก Top 5 housing_median_age

เติม SQL ในช่องว่าง


In [66]:
sql_cte = f"""
WITH base AS (
  SELECT housing_median_age
  FROM {table}
  WHERE median_house_value > 200000
),
agg AS (
  SELECT housing_median_age, COUNT(*) AS total_cases
  FROM base
  GROUP BY housing_median_age
)
SELECT *
FROM agg
ORDER BY total_cases DESC
LIMIT 5
"""

con.execute(sql_cte).df()


,housing_median_age,total_cases
0,52.0,688
1,36.0,318
2,35.0,288
3,16.0,269
4,25.0,246


## 10) Window Functions: TOP-N ต่อกลุ่ม

ใช้ `ROW_NUMBER()` หรือ `DENSE_RANK()` เพื่อหา Top median_house_value ต่อ housing_median_age


In [67]:
sql_window = f"""
WITH ranked AS (
  SELECT
    housing_median_age,
    median_house_value,
    ROW_NUMBER() OVER (PARTITION BY housing_median_age ORDER BY median_house_value DESC) AS rn
  FROM {table}
)
SELECT housing_median_age, median_house_value
FROM ranked
WHERE rn <= 3
ORDER BY housing_median_age, median_house_value DESC
LIMIT 10;
""".strip()

con.execute(sql_window).df()


,housing_median_age,median_house_value
0,1.0,191300.0
1,1.0,189200.0
2,2.0,500001.0
3,2.0,500001.0
4,2.0,500001.0
5,3.0,500001.0
6,3.0,500001.0
7,3.0,467600.0
8,4.0,500001.0
9,4.0,500001.0


### ✅ โจทย์ 5 (Hard): ROW_NUMBER vs DENSE_RANK
1) เปลี่ยน `ROW_NUMBER()` เป็น `DENSE_RANK()`  
2) ดูผลต่าง (โดยเฉพาะตอนคะแนนเท่ากัน)  
3) ตอบสั้น ๆ 2–3 บรรทัด: ต่างกันยังไง


In [68]:
# TODO: เปลี่ยนเป็น DENSE_RANK
sql_dense = f"""
WITH ranked AS (
  SELECT
    housing_median_age,
    median_house_value,
    DENSE_RANK() OVER (PARTITION BY housing_median_age ORDER BY median_house_value DESC) AS rk
  FROM {table}
)
SELECT housing_median_age, median_house_value, rk
FROM ranked
WHERE rk <= 3
ORDER BY housing_median_age, median_house_value DESC
LIMIT 10;
""".strip()

con.execute(sql_dense).df()


,housing_median_age,median_house_value,rk
0,1.0,191300.0,1
1,1.0,189200.0,2
2,2.0,500001.0,1
3,2.0,500001.0,1
4,2.0,500001.0,1
5,2.0,484100.0,2
6,2.0,450000.0,3
7,3.0,500001.0,1
8,3.0,500001.0,1
9,3.0,467600.0,2


## 11) โยงไป Spark SQL (Concept Bridge)

วันนี้เราใช้ DuckDB บนเครื่องเดียว แต่ SQL ที่เขียน “เกือบทั้งหมด” ย้ายไป Spark SQL ได้

### สิ่งที่เหมือน
- `SELECT/WHERE/GROUP BY/JOIN/CTE/WINDOW`

### สิ่งที่จะเพิ่มเมื่อเป็น Spark
- Distributed execution, partition, shuffle
- Lazy evaluation
- Physical plan + tuning (memory, spill, skew)

ตัวอย่าง (แนวคิด):


In [69]:
print("✅ วันนี้จบที่ DuckDB + SQL\nคาบถัดไป: Spark Architecture + Spark SQL บนข้อมูลใหญ่ขึ้น/หลายเครื่อง")


✅ วันนี้จบที่ DuckDB + SQL
คาบถัดไป: Spark Architecture + Spark SQL บนข้อมูลใหญ่ขึ้น/หลายเครื่อง


## 12) Checklist ส่งท้ายคาบ

- [ ] ใช้ LIMIT ทุกครั้งที่ preview
- [ ] ทำโจทย์ 1–5 อย่างน้อย 3 ข้อ
- [ ] รัน EXPLAIN แล้วแคปผล 1 รูป
- [ ] เขียน reflection 5–8 บรรทัด: Parquet ช่วยยังไง? SQL ช่วยยังไง?


# 12) Database Optimization & Performance Tricks (เทคนิคการเพิ่มประสิทธิภาพ Query)

การเขียน SQL ให้ได้ผลลัพธ์ที่ถูกต้องอย่างเดียวไม่พอ ในโลก Big Data เราต้องเขียนให้ **เร็ว** และ **ประหยัดทรัพยากร** ด้วย

### หลักการพื้นฐาน
1. **Select Only What You Need**: อย่าใช้ `SELECT *` ถ้าไม่จำเป็น เพราะการอ่านข้อมูลทุกคอลัมน์ (โดยเฉพาะใน Parquet/Columnar DB) เสียเวลามาก
2. **Filter Early**: ใช้ `WHERE` ให้เร็วที่สุด เพื่อลดจำนวน rows ก่อนที่จะนำไป JOIN หรือ Aggregate
3. **Understanding Storage**: รู้จักความต่างของ Row vs Column Store (ตามรูปภาพด้านบน)

### ตัวอย่าง: เปรียบเทียบความเร็ว
เราจะทดลอง Query แบบที่ "ไม่ดี" (Full Scan) เทียบกับแบบที่ "ดี" (Predicate Pushdown + Column Selection)

In [70]:
import time

# 1. Bad Query: SELECT * และ Filter ทีหลัง (จำลอง)
def run_bad_query():
    start = time.time()
    # ดึงมาทั้งหมดก่อน แล้วค่อยมา filter ใน Python Application (สมมติ)
    df = con.execute("SELECT * FROM housing_parquet").df()
    result = df[df['housing_median_age'] > 40]
    return time.time() - start

# 2. Good Query: Filter ใน Database Engine และเลือกแต่คอลัมน์ที่ใช้
def run_good_query():
    start = time.time()
    # ให้ DB Engine Filter ให้ (Predicate Pushdown) และเลือก Column แค่ที่ใช้
    con.execute("""
        SELECT median_house_value, housing_median_age
        FROM housing_parquet
        WHERE housing_median_age > 40
    """).df()
    return time.time() - start

time_bad = run_bad_query()
time_good = run_good_query()

print(f"❌ Bad Query Time:  {time_bad:.4f} sec")
print(f"✅ Good Query Time: {time_good:.4f} sec")
print(f"🚀 Speedup: {time_bad/time_good:.2f}x")

❌ Bad Query Time:  0.0110 sec
✅ Good Query Time: 0.0025 sec
🚀 Speedup: 4.37x


---
## 13) เริ่มต้นกับ Apache Spark & Spark SQL

### ทำไมต้องย้ายจาก DuckDB ไป Spark?

#### ข้อจำกัดของ DuckDB (Single-Node Processing)

DuckDB เป็น Engine ที่ยอดเยี่ยมสำหรับการวิเคราะห์ข้อมูลบนเครื่องเดียว แต่มีข้อจำกัด:

1. **ถูกจำกัดด้วย RAM**: ถ้าข้อมูลใหญ่กว่า RAM ของเครื่อง (เช่น ข้อมูล 100 GB แต่มี RAM แค่ 16 GB) จะประมวลผลไม่ได้หรือช้ามาก
2. **ใช้ CPU เครื่องเดียว**: แม้จะมี Multi-threading แต่ก็ใช้ได้แค่ CPU ของเครื่องเดียว (เช่น 8 cores)
3. **ไม่เหมาะกับข้อมูลแบบ Streaming**: DuckDB เน้น Batch Processing (ข้อมูลที่มีอยู่แล้ว) ไม่เหมาะกับข้อมูลแบบ Real-time

#### Apache Spark คืออะไร?

**Apache Spark** เป็น **Distributed Computing Engine** ที่ออกแบบมาเพื่อ:
- ประมวลผลข้อมูลขนาดใหญ่ (Big Data) ที่เกินความสามารถของเครื่องเดียว
- กระจายงานไปทำบนเครื่องหลายเครื่อง (Cluster) พร้อมกัน
- รองรับทั้ง Batch Processing และ Stream Processing

#### จุดเด่นของ Spark:

1. **In-Memory Processing**:
   - Spark เก็บข้อมูลในแรม (RAM) ของทุกเครื่องใน Cluster
   - เร็วกว่า Hadoop MapReduce (ที่เขียนลง Disk ทุกครั้ง) ถึง **100 เท่า**

2. **Distributed Computing**:
   - ถ้ามี Cluster 100 เครื่อง แต่ละเครื่อง 16 cores = ใช้ 1,600 cores พร้อมกัน
   - ข้อมูล 1 TB สามารถแบ่งเป็น 100 ส่วน ให้แต่ละเครื่องทำ 10 GB

3. **Fault Tolerance (ทนต่อความผิดพลาด)**:
   - ถ้าเครื่องใดเครื่องหนึ่งใน Cluster พัง Spark จะ Re-compute ส่วนนั้นอัตโนมัติ
   - ใช้เทคนิค **RDD (Resilient Distributed Dataset)** และ **DAG (Directed Acyclic Graph)**

4. **Lazy Evaluation**:
   - Spark ไม่ทำงานทันทีที่เราเขียนโค้ด
   - จะรอจนกว่าเราสั่ง **Action** (เช่น `.show()`, `.count()`) แล้วจึงทำทั้งหมดพร้อมกัน
   - ทำให้ Optimize ได้ดีกว่า (เช่น รวม Filter หลาย ๆ ตัวเป็นตัวเดียว)

5. **รองรับหลายภาษา**:
   - Python (PySpark)
   - Scala (ภาษาหลักของ Spark)
   - Java
   - R (SparkR)
   - **SQL (Spark SQL)** ← เราจะใช้ตัวนี้!

### Spark SQL: SQL บน Big Data

**Spark SQL** คือ Module ของ Spark ที่ให้เราใช้ SQL ในการ Query ข้อมูลบน Distributed System

#### ทำไม Spark SQL ถึงสำคัญ?

1. **Syntax เหมือน SQL ทั่วไป**:
   - คุณเขียน SQL เหมือนใน DuckDB, PostgreSQL, MySQL
   - ไม่ต้องเรียนรู้ภาษาใหม่ (เช่น Scala)

2. **ทำงานบน Distributed Data**:
   - SQL ที่เขียนจะถูกแปลงเป็น Spark Jobs
   - กระจายไปทำบนทุกเครื่องใน Cluster

3. **Optimize อัตโนมัติ**:
   - Spark มี **Catalyst Optimizer** ที่จะปรับ Query Plan ให้เร็วที่สุด
   - เช่น Push-down Filters, Predicate Pushdown, Join Optimization

4. **รองรับหลาย Data Source**:
   - CSV, Parquet, JSON, Avro
   - Hive Tables
   - JDBC (PostgreSQL, MySQL)
   - S3, HDFS, Azure Blob Storage

### เปรียบเทียบ DuckDB vs Spark SQL

| หัวข้อ | DuckDB | Spark SQL |
|--------|--------|-----------|
| **ขนาดข้อมูล** | MB - GB (สูงสุด ~100 GB) | GB - PB (ไม่จำกัด) |
| **จำนวนเครื่อง** | 1 เครื่อง | 1 - 1000+ เครื่อง |
| **ความเร็ว (ข้อมูลเล็ก)** | เร็วมาก | ช้ากว่า (Overhead จาก Distribution) |
| **ความเร็ว (ข้อมูลใหญ่)** | ไม่ไหว (Out of Memory) | เร็ว (Parallel Processing) |
| **Setup** | ง่าย (pip install) | ซับซ้อน (ต้องมี Cluster) |
| **Use Case** | Data Analysis บน Laptop | Production Big Data Pipeline |

### Concept: SQL Portability (ย้ายโค้ดได้ง่าย)

นี่คือเหตุผลที่ SQL ทรงพลัง:

```sql
-- SQL นี้ทำงานได้ทั้ง DuckDB และ Spark SQL!
SELECT
    housing_median_age,
    COUNT(*) as total_cases
FROM housing
WHERE housing_median_age > 10
GROUP BY housing_median_age
ORDER BY total_cases DESC
LIMIT 10
```

- **ข้อมูลเล็ก** (1 GB): รันบน DuckDB ในเครื่อง → ได้ผลใน 2 วินาที
- **ข้อมูลใหญ่** (1 TB): รันบน Spark Cluster → ได้ผลใน 30 วินาที
- **โค้ดเหมือนกัน 100%!**

### สถาปัตยกรรม Spark (แบบง่าย)

<img src="https://github.com/witsarutsarai12-Academic/128-356-Big-Data/blob/main/images/spark_arch.png?raw=1" width="700"/>



- **Driver**: เครื่องที่เราเขียนโค้ด (Jupyter Notebook)
- **Executor**: โปรแกรมที่รันบน Worker Node แต่ละเครื่อง
- **Task**: งานย่อย ๆ ที่แต่ละ Executor ทำ

### ตัวอย่างการทำงาน

สมมติเรามีไฟล์ `housing.parquet` ขนาด 100 GB บน S3:

1. เราเขียน SQL: `SELECT * FROM housing WHERE housing_median_age > 10`
2. Spark แบ่งไฟล์เป็น 1,000 ส่วน (Partitions)
3. ส่งแต่ละ Partition ไปให้ Worker Node ต่าง ๆ ทำ
4. แต่ละ Worker Filter เฉพาะ `housing_median_age > 10` ในส่วนของตัวเอง
5. รวมผลลัพธ์กลับมาที่ Driver
6. แสดงผล

**ผลลัพธ์**: งานที่ใช้เวลา 10 ชั่วโมงบนเครื่องเดียว → เหลือ 5 นาทีบน Cluster 100 เครื่อง!

### เมื่อไหร่ควรใช้ Spark?

✅ **ใช้ Spark เมื่อ**:
- ข้อมูลใหญ่กว่า RAM ของเครื่อง (> 50-100 GB)
- ต้องการประมวลผลข้อมูลแบบ Real-time (Streaming)
- ทำงานกับ Data Lake (S3, HDFS) ที่มีข้อมูล TB-PB
- ต้องการ Production Pipeline ที่ Scalable

❌ **ไม่ควรใช้ Spark เมื่อ**:
- ข้อมูลเล็ก (< 10 GB) → ใช้ DuckDB/Pandas เร็วกว่า
- ทำงานบน Laptop ส่วนตัว → Setup ยุ่งยาก
- ต้องการ Prototype เร็ว ๆ → DuckDB ง่ายกว่า


In [71]:
# ติดตั้ง PySpark (สำหรับการรันใน Colab/Local)
# PySpark คือ Python API สำหรับ Apache Spark
!pip -q install pyspark

# ตรวจสอบ version
import pyspark
print("PySpark version:", pyspark.__version__)


PySpark version: 4.0.1


In [72]:
from pyspark.sql import SparkSession

# สร้าง SparkSession (จุดเริ่มต้นของการใช้งาน Spark)
# SparkSession คือ Entry Point สำหรับการทำงานกับ Spark
# ในโหมด Local จะใช้ CPU ของเครื่องเราเอง (ไม่ได้เชื่อมกับ Cluster จริง)

spark = SparkSession.builder \
    .appName("Week3_SparkSQL") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("✅ Spark Session สร้างสำเร็จ!")
print("Spark Version:", spark.version)
print("Spark UI:", spark.sparkContext.uiWebUrl)  # ดู Monitoring UI (ถ้ารันบน Local)


✅ Spark Session สร้างสำเร็จ!
Spark Version: 4.0.1
Spark UI: http://3c2598705ed5:4040


In [73]:
# ---------------------------------------------------------
# 1) อ่านข้อมูล (Load Data)
# ---------------------------------------------------------
if os.path.exists("housing.parquet"):
    df_spark = spark.read.parquet("housing.parquet")
    print("✅ อ่านจาก Parquet Files (เร็ว/เล็ก)")
else:
    df_spark = spark.read.option("header", "true").csv("sample_data/california_housing_train.csv")
    print("✅ อ่านจาก CSV Files (ช้ากว่า)")

# ---------------------------------------------------------
# 2) สร้าง Temporary View เพื่อใช้ Spark SQL
# ---------------------------------------------------------
df_spark.createOrReplaceTempView("housing")
print("✅ Created Temp View: 'housing'")
df_spark.printSchema()

# ---------------------------------------------------------
# 3) เขียน Spark SQL Query
# ---------------------------------------------------------
spark_sql = """
SELECT
    housing_median_age,
    AVG(median_house_value) AS avg_price,
    COUNT(*) AS count
FROM housing
GROUP BY housing_median_age
ORDER BY avg_price DESC
LIMIT 5
"""

# ---------------------------------------------------------
# 4) Run & Show Result
# ---------------------------------------------------------
print("\n--- Result ---")
spark.sql(spark_sql).show()


✅ อ่านจาก Parquet Files (เร็ว/เล็ก)
✅ Created Temp View: 'housing'
root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)


--- Result ---
+------------------+------------------+-----+
|housing_median_age|         avg_price|count|
+------------------+------------------+-----+
|              52.0| 277368.2005703422| 1052|
|              51.0|        244850.125|   32|
|               3.0|239450.04347826086|   46|
|               4.0| 230054.1055900621|  161|
|               2.0|229438.83673469388|   49|
+------------------+------------------+-----+



# Individual Assignment – End of Week 3

**Dataset**: Online Retail (UCI Machine Learning Repository)  
**ขนาด**: ~500k rows  
**รูปแบบ**: CSV → SQL analytics

## 🎯 เป้าหมายของงาน

ให้นักศึกษา **จัดการข้อมูลเองตั้งแต่ต้น** และตอบคำถามเชิงวิเคราะห์ด้วย SQL

- ✅ **เน้น** การตั้งคำถามกับข้อมูล + การสรุปเชิงเหตุผล

---

### 🔹 Part 0: ทำความเข้าใจข้อมูล (Warm-up)

ให้นักศึกษาตอบเป็นข้อความสั้น ๆ (ไม่ต้องใช้ SQL ก็ได้)

1. Dataset นี้เกี่ยวกับอะไร
2. หนึ่งแถว (row) แทน “เหตุการณ์” อะไร
3. คอลัมน์ใดน่าจะเป็น ตัวระบุสินค้า / ลูกค้า / เวลา

> 🎯 **จุดประสงค์**: วัดว่า “อ่าน data description เป็นไหม”

### 🔹 Part 1: Data Exploration ด้วย SQL (พื้นฐาน)

ให้นักศึกษาสร้าง query เพื่อตอบคำถามต่อไปนี้

1. Dataset นี้มีทั้งหมดกี่แถว
2. มีทั้งหมดกี่ประเทศ
3. มีสินค้าทั้งหมดกี่รายการ (distinct product)
4. มีลูกค้าทั้งหมดกี่ราย (CustomerID ที่ไม่เป็น NULL)

> 💡 **Hint**: `COUNT(*)`, `COUNT(DISTINCT ...)`, `WHERE`

### 🔹 Part 2: วิเคราะห์ยอดขาย (Core Analytics)

ให้นศ. คิดว่า “ยอดขาย” ควรนิยามอย่างไร

1. **ยอดขายรวมทั้งหมด** (Total Revenue)
2. **ยอดขายรวมต่อประเทศ** (Top 5 ประเทศแรก)
3. **สินค้าที่ขายได้ยอดรวมสูงสุด 5 อันดับแรก**
4. **ลูกค้าที่มียอดซื้อรวมสูงสุด 5 อันดับแรก**

> 💡 **Hint**: `quantity * unitprice`, `GROUP BY`, `ORDER BY`

### 🔹 Part 3: เวลา & พฤติกรรมการซื้อ (Thinking Section)

1. เดือนใดมียอดขายรวมสูงที่สุด
2. วันในสัปดาห์ใดมีจำนวน invoice มากที่สุด
3. ชั่วโมงใดของวันมีการออก invoice มากที่สุด

> 🎯 **จุดนี้จะบังคับให้นศ.:**  
> - แปลง datetime  
> - เข้าใจว่า “เวลา” มีผลต่อ business insight

### 🔹 Part 4: Data Quality & Anomaly (สำคัญมาก)

1. มีแถวข้อมูลที่ CustomerID เป็น NULL กี่แถว
2. มี invoice ที่ quantity ติดลบหรือไม่
3. หากลบข้อมูลผิดปกติออก (quantity ≤ 0 หรือ price ≤ 0)  
   **ยอดขายรวมเปลี่ยนไปหรือไม่ อย่างไร**

> 🎯 **ตรงนี้คือ สะพานไปบท Data Quality (Week 10)**  
> เด็กจะเริ่มรู้ว่า “ข้อมูลจริงไม่สวย”

### 🔹 Part 5: คำถามเชิงวิเคราะห์ (Open-ended แต่บังคับใช้ SQL)

ให้นักศึกษาเลือก **อย่างน้อย 2 ข้อ**

1. ประเทศใด “มูลค่าต่อ invoice” สูงที่สุด
2. สินค้าใดถูกซื้อซ้ำบ่อยที่สุด
3. ลูกค้าส่วนใหญ่ซื้อของช่วงเวลาใด
4. มี pattern การซื้อที่น่าสนใจอะไรบ้างจากข้อมูลนี้

> ✨ **คะแนนส่วนนี้ให้จาก**:  
> - ความสมเหตุสมผล  
> - SQL ที่ใช้  
> - การอธิบาย insight

In [48]:
import requests

# URL ของไฟล์ Online Retail.csv
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
output_path = "Online_Retail.xlsx"

# ดาวน์โหลดไฟล์ XLSX
print(f"Downloading {url} to {output_path}...")
response = requests.get(url)
with open(output_path, 'wb') as f:
    f.write(response.content)
print("Download complete.")

import pandas as pd

df_excel = pd.read_excel(output_path)
RETAIL_CSV_PATH = "Online_Retail.csv"
df_excel.to_csv(RETAIL_CSV_PATH, index=False)
print(f"Converted {output_path} to {RETAIL_CSV_PATH}")


Download complete.
Converted Online_Retail.xlsx to Online_Retail.csv


In [80]:
import requests
import pandas as pd
import duckdb
import time
import os

if 'con' not in globals():
    con = duckdb.connect(database=':memory:', read_only=False)
    print("DuckDB connection 'con' created.")
else:
    print("DuckDB connection 'con' already exists.")

# Download data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
output_path = "Online_Retail.xlsx"

print(f"Downloading {url} to {output_path}...")
response = requests.get(url)
response.raise_for_status()
with open(output_path, 'wb') as f:
    f.write(response.content)
print("Download complete.")

# Convert XLSX to CSV
df_excel = pd.read_excel(output_path)
RETAIL_CSV_PATH = "Online_Retail.csv"
df_excel.to_csv(RETAIL_CSV_PATH, index=False)
print(f"Converted {output_path} to {RETAIL_CSV_PATH}")

# Create DuckDB view
con.execute(f"""
    CREATE OR REPLACE VIEW retail AS
    SELECT * FROM read_csv_auto('{RETAIL_CSV_PATH}');
""")
print("✅ created view: retail")
print("Showing first 5 rows of 'retail' view:")
display(con.execute("SELECT * FROM retail LIMIT 5").df())


DuckDB connection 'con' already exists.
Download complete.
Converted Online_Retail.xlsx to Online_Retail.csv
✅ created view: retail
Showing first 5 rows of 'retail' view:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### 🔹 Part 0: ทำความเข้าใจข้อมูล (Warm-up)

1.  **Dataset นี้เกี่ยวกับอะไร: ข้อมูลการซื้อขายสินค้าออนไลน์
2.  **หนึ่งแถว (row) แทน “เหตุการณ์” อะไร:** การซื้อสินค้า 1 ชิ้นในหนึ่งใบเสร็จ
3.  **คอลัมน์ใดน่าจะเป็น ตัวระบุสินค้า / ลูกค้า / เวลา:** _**สินค้า:** `StockCode`, `Description` **ลูกค้า:** `CustomerID` **เวลา:** `InvoiceDate`_


### 🔹 Part 1: Data Exploration ด้วย SQL (พื้นฐาน)


In [81]:
# 1. Dataset นี้มีทั้งหมดกี่แถว
sql_part1_q1 = """
SELECT COUNT(*) AS total_rows
FROM retail;
"""

# 2. มีทั้งหมดกี่ประเทศ (นับค่าที่ไม่ซ้ำกัน)
sql_part1_q2 = """
SELECT COUNT(DISTINCT Country) AS total_countries
FROM retail;
"""

# 3. มีสินค้าทั้งหมดกี่รายการ (distinct StockCode)
sql_part1_q3 = """
SELECT COUNT(DISTINCT StockCode) AS total_distinct_products
FROM retail;
"""

# 4. มีลูกค้าทั้งหมดกี่ราย (CustomerID ที่ไม่เป็น NULL)
sql_part1_q4 = """
SELECT COUNT(DISTINCT CustomerID) AS total_distinct_customers
FROM retail
WHERE CustomerID IS NOT NULL;
"""

print("คำตอบสำหรับ Part 1:")
display(con.execute(sql_part1_q1).df())
display(con.execute(sql_part1_q2).df())
display(con.execute(sql_part1_q3).df())
display(con.execute(sql_part1_q4).df())


คำตอบสำหรับ Part 1:


,total_rows
0,541909


,total_countries
0,38


,total_distinct_products
0,4070


,total_distinct_customers
0,4372


### 🔹 Part 2: วิเคราะห์ยอดขาย (Core Analytics)


In [86]:
# 1. ยอดขายรวมทั้งหมด (Total Revenue)
sql_part2_q1 = """
SELECT SUM(Quantity * UnitPrice) AS Total_Revenue
FROM retail;
"""

# 2. ยอดขายรวมต่อประเทศ (Top 5 ประเทศแรก)
sql_part2_q2 = """
SELECT Country, SUM(Quantity * UnitPrice) AS Total_Revenue_by_Country
FROM retail
WHERE Country IS NOT NULL AND Quantity > 0 AND UnitPrice > 0
GROUP BY Country
ORDER BY Total_Revenue_by_Country DESC
LIMIT 5;
"""

# 3. สินค้าที่ขายได้ยอดรวมสูงสุด 5 อันดับแรก (Description เป็นชื่อสินค้า)
sql_part2_q3 = """
SELECT Description, SUM(Quantity * UnitPrice) AS Total_Revenue_by_Product
FROM retail
WHERE Description IS NOT NULL AND Quantity > 0 AND UnitPrice > 0
GROUP BY Description
ORDER BY Total_Revenue_by_Product DESC
LIMIT 5;
"""

# 4. ลูกค้าที่มียอดซื้อรวมสูงสุด 5 อันดับแรก (CustomerID ที่ไม่เป็น NULL)
sql_part2_q4 = """
SELECT CustomerID, SUM(Quantity * UnitPrice) AS Total_Spending_by_Customer
FROM retail
WHERE CustomerID IS NOT NULL AND Quantity > 0 AND UnitPrice > 0
GROUP BY CustomerID
ORDER BY Total_Spending_by_Customer DESC
LIMIT 5;
"""

print("ยอดขายรวมทั้งหมด:")
display(con.execute(sql_part2_q1).df())

print("\n ยอดขายรวมต่อประเทศ (Top 5):")
display(con.execute(sql_part2_q2).df())

print("\n สินค้าที่ขายได้ยอดรวมสูงสุด 5 อันดับแรก:")
display(con.execute(sql_part2_q3).df())

print("\n ลูกค้าที่มียอดซื้อรวมสูงสุด 5 อันดับแรก:")
display(con.execute(sql_part2_q4).df())


ยอดขายรวมทั้งหมด:


,Total_Revenue
0,9.747748e+06



 ยอดขายรวมต่อประเทศ (Top 5):


,Country,Total_Revenue_by_Country
0,United Kingdom,9.025222e+06
1,Netherlands,2.854463e+05
2,EIRE,2.834540e+05
3,Germany,2.288671e+05
4,France,2.097151e+05



 สินค้าที่ขายได้ยอดรวมสูงสุด 5 อันดับแรก:


,Description,Total_Revenue_by_Product
0,DOTCOM POSTAGE,206248.77
1,REGENCY CAKESTAND 3 TIER,174484.74
2,"PAPER CRAFT , LITTLE BIRDIE",168469.60
3,WHITE HANGING HEART T-LIGHT HOLDER,106292.77
4,PARTY BUNTING,99504.33



 ลูกค้าที่มียอดซื้อรวมสูงสุด 5 อันดับแรก:


,CustomerID,Total_Spending_by_Customer
0,14646.0,280206.02
1,18102.0,259657.30
2,17450.0,194550.79
3,16446.0,168472.50
4,14911.0,143825.06


### 🔹 Part 3: เวลา & พฤติกรรมการซื้อ (Thinking Section)


In [83]:
# 1. เดือนใดมียอดขายรวมสูงที่สุด
sql_part3_q1 = """
SELECT
    STRFTIME(InvoiceDate, '%Y-%m') AS sale_month,
    SUM(Quantity * UnitPrice) AS Total_Revenue
FROM retail
WHERE Quantity > 0 AND UnitPrice > 0
GROUP BY sale_month
ORDER BY Total_Revenue DESC
LIMIT 5;
"""

# 2. วันในสัปดาห์ใดมีจำนวน invoice มากที่สุด (วันจันทร์-อาทิตย์)
sql_part3_q2 = """
SELECT
    STRFTIME(InvoiceDate, '%w') AS day_of_week_num,
    CASE STRFTIME(InvoiceDate, '%w')
        WHEN '0' THEN 'Sunday'
        WHEN '1' THEN 'Monday'
        WHEN '2' THEN 'Tuesday'
        WHEN '3' THEN 'Wednesday'
        WHEN '4' THEN 'Thursday'
        WHEN '5' THEN 'Friday'
        WHEN '6' THEN 'Saturday'
    END AS day_of_week_name,
    COUNT(DISTINCT InvoiceNo) AS Total_Invoices
FROM retail
WHERE Quantity > 0 AND UnitPrice > 0
GROUP BY day_of_week_num, day_of_week_name
ORDER BY Total_Invoices DESC
LIMIT 7;
"""

# 3. ชั่วโมงใดของวันมีการออก invoice มากที่สุด
sql_part3_q3 = """
SELECT
    STRFTIME(InvoiceDate, '%H') AS sale_hour,
    COUNT(DISTINCT InvoiceNo) AS Total_Invoices
FROM retail
WHERE Quantity > 0 AND UnitPrice > 0
GROUP BY sale_hour
ORDER BY Total_Invoices DESC
LIMIT 5;
"""

print("\n--- คำตอบสำหรับ Part 3 ---")
print("1. เดือนใดมียอดขายรวมสูงที่สุด:")
display(con.execute(sql_part3_q1).df())

print("\n2. วันในสัปดาห์ใดมีจำนวน invoice มากที่สุด:")
display(con.execute(sql_part3_q2).df())

print("\n3. ชั่วโมงใดของวันมีการออก invoice มากที่สุด:")
display(con.execute(sql_part3_q3).df())



--- คำตอบสำหรับ Part 3 ---
1. เดือนใดมียอดขายรวมสูงที่สุด:


,sale_month,Total_Revenue
0,2011-11,1509496.330
1,2011-10,1154979.300
2,2011-09,1058590.172
3,2010-12,823746.140
4,2011-05,770536.020



2. วันในสัปดาห์ใดมีจำนวน invoice มากที่สุด:


,day_of_week_num,day_of_week_name,Total_Invoices
0,4,Thursday,4246
1,3,Wednesday,3690
2,2,Tuesday,3554
3,5,Friday,3140
4,1,Monday,3126
5,0,Sunday,2204



3. ชั่วโมงใดของวันมีการออก invoice มากที่สุด:


,sale_hour,Total_Invoices
0,12,3220
1,13,2753
2,14,2457
3,11,2396
4,10,2361


### 🔹 Part 4: Data Quality & Anomaly (สำคัญมาก)


In [84]:
# 1. มีแถวข้อมูลที่ CustomerID เป็น NULL กี่แถว
sql_part4_q1 = """
SELECT COUNT(*) AS rows_with_null_customerid
FROM retail
WHERE CustomerID IS NULL;
"""

# 2. มี invoice ที่ Quantity ติดลบหรือไม่
sql_part4_q2_quantity_negative = """
SELECT COUNT(*) AS rows_with_negative_quantity
FROM retail
WHERE Quantity <= 0;
"""

# 3. มี invoice ที่ UnitPrice ติดลบหรือไม่
sql_part4_q2_price_negative = """
SELECT COUNT(*) AS rows_with_negative_price
FROM retail
WHERE UnitPrice <= 0;
"""

# ยอดขายรวมหลังจากลบข้อมูลผิดปกติออก (quantity <= 0 หรือ price <= 0)
sql_part4_q3_filtered_revenue = """
SELECT SUM(Quantity * UnitPrice) AS Total_Revenue_After_Filtering
FROM retail
WHERE Quantity > 0 AND UnitPrice > 0;
"""

print("\n--- คำตอบสำหรับ Part 4 ---")
print("1. จำนวนแถวที่ CustomerID เป็น NULL:")
display(con.execute(sql_part4_q1).df())

print("\n2. จำนวนแถวที่ Quantity <= 0:")
display(con.execute(sql_part4_q2_quantity_negative).df())

print("\n3. จำนวนแถวที่ UnitPrice <= 0:")
display(con.execute(sql_part4_q2_price_negative).df())

print("\n4. ยอดขายรวมก่อนกรอง (จาก Part 2):")
display(con.execute(sql_part2_q1).df())

print("\n5. ยอดขายรวมหลังจากลบข้อมูลผิดปกติ (Quantity > 0 และ UnitPrice > 0):")
display(con.execute(sql_part4_q3_filtered_revenue).df())

original_revenue = con.execute(sql_part2_q1).df().iloc[0]['Total_Revenue']
filtered_revenue = con.execute(sql_part4_q3_filtered_revenue).df().iloc[0]['Total_Revenue_After_Filtering']

print(f"\nยอดขายรวมเปลี่ยนไปหรือไม่ อย่างไร: {(original_revenue - filtered_revenue):.2f}")
if original_revenue > filtered_revenue:
    print("ยอดขายลดลงหลังจากกรองข้อมูลที่ไม่ถูกต้องออก.")
elif original_revenue < filtered_revenue:
    print("ยอดขายเพิ่มขึ้นหลังจากกรองข้อมูลที่ไม่ถูกต้องออก (เนื่องจาก Quantity ติดลบถูกนำมาหักลบในยอดรวมเดิม).")
else:
    print("ยอดขายไม่เปลี่ยนแปลง.")



--- คำตอบสำหรับ Part 4 ---
1. จำนวนแถวที่ CustomerID เป็น NULL:


,rows_with_null_customerid
0,135080



2. จำนวนแถวที่ Quantity <= 0:


,rows_with_negative_quantity
0,10624



3. จำนวนแถวที่ UnitPrice <= 0:


...


4. ยอดขายรวมก่อนกรอง (จาก Part 2):


,Total_Revenue
0,9.747748e+06



5. ยอดขายรวมหลังจากลบข้อมูลผิดปกติ (Quantity > 0 และ UnitPrice > 0):


,Total_Revenue_After_Filtering
0,1.066668e+07



ยอดขายรวมเปลี่ยนไปหรือไม่ อย่างไร: -918936.61
ยอดขายเพิ่มขึ้นหลังจากกรองข้อมูลที่ไม่ถูกต้องออก (เนื่องจาก Quantity ติดลบถูกนำมาหักลบในยอดรวมเดิม).


### 🔹 Part 5: คำถามเชิงวิเคราะห์ (Open-ended แต่บังคับใช้ SQL)

**เลือก 2 ข้อ:**
1. ประเทศใด “มูลค่าต่อ invoice” สูงที่สุด
2. สินค้าใดถูกซื้อซ้ำบ่อยที่สุด


In [85]:
# 1. ประเทศใด “มูลค่าต่อ invoice” สูงที่สุด
sql_part5_q1 = """
WITH InvoiceTotal AS (
    SELECT
        InvoiceNo,
        Country,
        SUM(Quantity * UnitPrice) AS InvoiceRevenue
    FROM retail
    WHERE Quantity > 0 AND UnitPrice > 0
    GROUP BY InvoiceNo, Country
)
SELECT
    Country,
    AVG(InvoiceRevenue) AS AverageInvoiceValue
FROM InvoiceTotal
GROUP BY Country
ORDER BY AverageInvoiceValue DESC
LIMIT 5;
"""

print("\n--- คำตอบสำหรับ Part 5 (Q1) ---")
print("1. ประเทศใด 'มูลค่าต่อ invoice' สูงที่สุด:")
display(con.execute(sql_part5_q1).df())

# 2. สินค้าใดถูกซื้อซ้ำบ่อยที่สุด
sql_part5_q2 = """
SELECT
    Description,
    COUNT(DISTINCT CustomerID) AS NumberOfCustomers,
    COUNT(*) AS TotalPurchaseCount
FROM retail
WHERE Description IS NOT NULL AND Quantity > 0
GROUP BY Description
ORDER BY TotalPurchaseCount DESC
LIMIT 5;
"""

print("\n--- คำตอบสำหรับ Part 5 (Q2) ---")
print("2. สินค้าใดถูกซื้อซ้ำบ่อยที่สุด:")
display(con.execute(sql_part5_q2).df())

# Insight:
# (คำตอบของคุณ:)
# 1. Singapore และ Netherlands มีมูลค่าต่อ invoice สูง
# 2. WHITE HANGING HEART T-LIGHT HOLDER และ JUMBO BAG RED RETROSPOT ถูกซื้อบ่อยที่สุด



--- คำตอบสำหรับ Part 5 (Q1) ---
1. ประเทศใด 'มูลค่าต่อ invoice' สูงที่สุด:


,Country,AverageInvoiceValue
0,Singapore,3039.898571
1,Netherlands,3036.663191
2,Australia,2430.198421
3,Japan,1969.282632
4,Lebanon,1693.880000



--- คำตอบสำหรับ Part 5 (Q2) ---
2. สินค้าใดถูกซื้อซ้ำบ่อยที่สุด:


,Description,NumberOfCustomers,TotalPurchaseCount
0,WHITE HANGING HEART T-LIGHT HOLDER,856,2327
1,JUMBO BAG RED RETROSPOT,635,2115
2,REGENCY CAKESTAND 3 TIER,881,2019
3,PARTY BUNTING,708,1707
4,LUNCH BAG RED RETROSPOT,532,1594


In [49]:
# สร้าง DuckDB VIEW จากไฟล์ CSV ที่ดาวน์โหลดมา
# ตรวจสอบให้แน่ใจว่าได้รัน cell ที่ติดตั้ง DuckDB และสร้าง 'con' object ไว้แล้ว
if 'con' not in globals():
    print("⚠️ DuckDB connection 'con' not found. Please run the setup cells first.")
else:
    con.execute(f"""
        CREATE OR REPLACE VIEW retail AS
        SELECT * FROM read_csv_auto('{RETAIL_CSV_PATH}');
    """)
    print("✅ created view: retail")
    print("แสดง 5 แถวแรกของ view 'retail':")
    display(con.execute("SELECT * FROM retail LIMIT 5").df())


✅ created view: retail
แสดง 5 แถวแรกของ view 'retail':


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
